In [23]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
from gpt import SingleHeadAttention, MultiHeadAttention, FeedForward, Block, GPT

In [25]:
# hyperparameters, as in gpt.py
batch_size = 8 
block_size = 16 
max_iters = 4000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 32
n_head = 2
n_layer = 2
dropout = 0.0

In [26]:
torch.manual_seed(2525)

In [27]:
with open('tiny-shakespeare.txt', 'r', encoding='utf-8') as f:
    text=f.read()

In [28]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [29]:
len(text)

1115394

In [30]:
chars = sorted(list(set(text)))
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [31]:
vocab_size = len(chars)
vocab_size

65

In [32]:
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [33]:
print(encode("Sword in Mouth, Fire Eyes"))
print(decode(encode("Hello, World!")))

[31, 61, 53, 56, 42, 1, 47, 52, 1, 25, 53, 59, 58, 46, 6, 1, 18, 47, 56, 43, 1, 17, 63, 43, 57]
Hello, World!


In [34]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [35]:
print(data[:200])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [36]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [37]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

In [38]:
xb, yb = get_batch('train')

In [39]:
print("inputs:")
xb.shape, xb

inputs:


(torch.Size([8, 16]),
 tensor([[ 1, 44, 56, 53, 51,  1, 58, 46, 43,  1, 58, 53, 54,  1, 58, 53],
         [ 6,  1, 54, 43, 39, 41, 43,  6,  0, 28, 56, 53, 59, 42,  1, 57],
         [40, 53, 63,  6,  0, 21, 44,  1, 21,  1, 51, 39, 63,  1, 43, 60],
         [46, 63,  1, 46, 53, 59, 52, 42, 57,  1, 57, 46, 39, 50, 50,  1],
         [43,  6,  0, 25, 63,  1, 39, 40, 57, 53, 50, 59, 58, 43,  1, 54],
         [42, 63, 12,  0,  0, 28, 13, 33, 24, 21, 26, 13, 10,  0, 35, 46],
         [63,  1, 50, 53, 56, 42, 57,  6,  0, 13, 50, 58, 46, 53, 59, 45],
         [26, 30, 37,  1, 34, 21, 10,  0, 35, 46, 39, 58,  1, 58, 47, 58]]))

In [40]:
print("targets:")
yb.shape, yb

targets:


(torch.Size([8, 16]),
 tensor([[44, 56, 53, 51,  1, 58, 46, 43,  1, 58, 53, 54,  1, 58, 53,  1],
         [ 1, 54, 43, 39, 41, 43,  6,  0, 28, 56, 53, 59, 42,  1, 57, 43],
         [53, 63,  6,  0, 21, 44,  1, 21,  1, 51, 39, 63,  1, 43, 60, 43],
         [63,  1, 46, 53, 59, 52, 42, 57,  1, 57, 46, 39, 50, 50,  1, 51],
         [ 6,  0, 25, 63,  1, 39, 40, 57, 53, 50, 59, 58, 43,  1, 54, 53],
         [63, 12,  0,  0, 28, 13, 33, 24, 21, 26, 13, 10,  0, 35, 46, 39],
         [ 1, 50, 53, 56, 42, 57,  6,  0, 13, 50, 58, 46, 53, 59, 45, 46],
         [30, 37,  1, 34, 21, 10,  0, 35, 46, 39, 58,  1, 58, 47, 58, 50]]))

In [41]:
@torch.no_grad()
def estimate_loss():
    out:dict = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zero(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [55]:
class Bigram(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embd_table = nn.Embedding(vocab_size, vocab_size)
        self.position_embd_table = nn.Embedding(block_size, vocab_size)
        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.layernormf = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None) -> torch.Tensor:
        b, t = idx.shape
        token_embd = self.token_embd_table(idx)
        if t >= block_size:
            t = t-1
        pos_embd = self.position_embd_table(torch.arange(t, device=device))
        x = token_embd + pos_embd
        x = self.blocks(x)
        x = x.view(-1, 32)
        x = self.layernormf(x)
        logits = self.lm_head(x)
        if targets is None:
            loss = None
        else: 
            b, t, c = logits.shape
            logits = logits.view(b*t, c)
            targets = targets.view(b*t)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]        
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) 
            idx = torch.cat((idx, idx_next), dim=1) 
        return idx